In [8]:
import pandas as pd

</p>
0 - русский
1 - белорусский
2 - украинский
3 - болгарский
4 - македонский
5 - сербский
<p>

In [ ]:
languages_dict = {'rus' : 0, 'bel' : 1, 'ukr' : 2, 'bul' : 3, 'mac' : 4, 'srp' : 5}

In [2]:
df = pd.read_csv('./task2-dev.csv')

In [3]:
df

,language,text
0,0,Олег Шилов самостоятельно сделал мачту
1,0,Олег Шилов учился на факультете
2,0,Из-за пропуска военных лагерей был
3,0,Олег Шилов стал работать со
4,0,На европейских чемпионатах Хопёрский был
5,0,Впоследствии многие тренеры спрашивали у
6,0,Эта победа стала первой для
7,0,Экспресс-сообщение по линии не осуществляется
8,0,Между этой станцией и следующей
9,0,Виной всему стало неудовлетворительное техниче...


In [27]:
import goslate

In [8]:
import os

In [30]:
import detectlanguage

detectlanguage.configuration.api_key = "7f5517f2af159462e0dfc8aab23ae6ec"

In [31]:
text = []
language = []
with open('./test_no_labels.txt', 'r', encoding='utf-8') as f:
    for line in f:
        text.append(line)
        language.append(detectlanguage.simple_detect(line))

IndexError: list index out of range

In [34]:
X_test = pd.DataFrame(text)

In [40]:
language.append("idk!")

In [41]:
X_test['language'] = language

In [58]:
X_test['text'] = X_test[0].apply(lambda x: x[:-1])

In [60]:
X_test = X_test.drop(labels=[0], axis=1)

In [61]:
X_test

,language,text
1,mk,Ова се двете вообичаени варијанти
2,sr,"Науме, во режија на Благојче"
3,be,“ Белавежы ” добра паказала
4,la,"Впрочем, Homo supersapiens - это"
5,mk,припадник на македонските елитни единици
6,ru,никогда не оказываются в хосписах
7,mk,му се заблагодари на премиерот
8,sr,"била раније, али је"
9,bg,на ОССЕ в Хърватия публикуваха
10,be,складае ад 70 да 86


In [62]:
X_test.to_csv('test.csv')